In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from renewableopt.data import DATASET_DIR
from renewableopt.peak_id import assert_int

In [2]:
filename = DATASET_DIR / 'wind_data' / 'wind.csv'

In [3]:
wind_power = np.loadtxt(filename, delimiter=",", skiprows=1)
wind_power_with_time = np.loadtxt(DATASET_DIR / 'wind_data' / 'wind_with_time.csv', delimiter=",", skiprows=1)

In [4]:
dt = 30  # min
min_per_day = 60 * 24
num_days = assert_int(len(wind_power) * dt / min_per_day)

In [5]:
assert_int(min_per_day / dt)

48

In [6]:
def to_datetime(row):
    return np.datetime64(
        f"{row[0]:4.0f}-{row[1]:0>2.0f}-{row[2]:0>2.0f}T{row[3]:0>2.0f}:{row[4]:0>2.0f}"
    )

In [7]:
times = np.array([to_datetime(row) for row in wind_power_with_time])

In [8]:
def insert_feb_29(times, wind_power):
    missing = np.where(np.diff(times) != np.timedelta64(30))
    times_by_day = times.reshape(num_days, -1)
    wind_power_by_day = wind_power.reshape(num_days, -1)
    feb_28_ind = np.argwhere(times_by_day == times[missing])[0, 0]
    
    # Copy feb 28 data to feb 29
    new_times = np.insert(times_by_day, feb_28_ind + 1,
              times_by_day[feb_28_ind] + np.timedelta64(24, 'h'), axis=0)
    new_wind_power = np.insert(wind_power_by_day, feb_28_ind + 1,
                               wind_power_by_day[feb_28_ind], axis=0)
    return new_times.flatten(), new_wind_power.flatten()
    

In [9]:

times, wind_power = insert_feb_29(times, wind_power)

In [10]:
assert np.all(np.diff(times) == np.timedelta64(30))

In [11]:
# Cap wind power output per turbine at rated power
P_rated = 2.3
wind_power = np.clip(wind_power, None, P_rated)
# Normalize by rated power so that data is "per unit": MW (active) per MW (installed)
# rather than per installed turbine
wind_pu = wind_power / P_rated

In [12]:
wind_pu * P_rated

array([2.3       , 2.3       , 2.21276809, ..., 2.3       , 2.3       ,
       2.3       ])

In [13]:
time_min = (times - times[0]).astype('timedelta64[m]').astype(float)

In [15]:
np.savez_compressed(DATASET_DIR / 'wind_data' / "wind_corrected.npz", time_min=time_min, wind_pu=wind_pu)

In [ ]:
print(times_by_day[feb_28_ind, 0])
print(times_by_day[feb_28_ind + 1, 0])

In [ ]:
times_by_day[feb_28_ind, 0] + np.timedelta64(24, 'h')

In [ ]:
np.savez_compressed()

In [ ]:
times_by_day[feb_28_ind] + np.timedelta64(24, 'h')

In [ ]:
new_times_by_day[feb_28_ind][0]

In [ ]:
new_times_by_day[feb_28_ind + 1][0]

In [ ]:
new_times = new_times_by_day.flatten()
assert np.all(np.diff(new_times) == np.timedelta64(30))

In [ ]:
np.max(np.diff(new_times))

In [ ]:
plt.plot(np.diff(new_times))

In [ ]:
plt.plot(np.diff(new_times_by_day))

In [ ]:
new_times_by_day.shape

In [ ]:
np.max(np.diff(new_times))

In [ ]:
times_by_day

In [ ]:
times[missing[0] + 1]

In [ ]:
from renewableopt.peak_id import timedelta

In [ ]:
timedelta(times)

In [ ]:
f"{wind_power_with_time[0, 1]:0>2.0f}"

In [ ]:
to_datetime(wind_power_with_time[2]).astype("datetime64[s]").dtype

In [ ]:
for i, row in enumerate(wind_power_with_time):
    times[i] = np.datetime64(
        f"{row[0]:4.0f}-{row[1]:0>2.0f}-{row[2]:0>2.0f}T{row[3]:0>2.0f}:{row[4]:0>2.0f}"
    )

In [ ]:
np.datetime64(
    "2012-01-02T12:30"
)